In [ ]:
from langchain.llms import OpenAI
import openai
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


In [ ]:
import os 
openai_api_key= os.environ["OPENAI_API_KEY"]
huggingfacehub_api_token= os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [ ]:
llm= OpenAI(model_name="gpt-3.5-turbo",temperature=0.6)
print(llm)

In [ ]:
text = "what is the capital of Australia?"
print(llm.predict(text))

In [ ]:
from langchain import HuggingFaceHub
llm_hugingface = HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0.5,"max_length":64})

In [ ]:
text= "Can you rite a poem about AI ?"
output = llm_hugingface.predict(text)

print(output)

In [ ]:
output=llm.predict(text)


### Prompt Templates And LLM chain

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
prompt_template = PromptTemplate(
                                 input_variables=['country'],
                                 template = "tell me the capital of this {country}"
                                )
# prompt_template.format(country="Nepal")

In [ ]:
llm.predict(prompt_template=prompt_template,text=['India'])

In [ ]:
from langchain.chains import LLMChain

In [ ]:
chain = LLMChain(llm=llm,prompt=prompt_template)
chain.run('Nepal')

### Combining Multiple Chains Using SImple Sequential Chain

In [ ]:
capital_prompt = PromptTemplate(
    input_variables=['country'],
    template="tell me the capital of {country}"
    )
capital_chain= LLMChain(llm=llm,prompt=capital_prompt)

In [ ]:
famous_template = PromptTemplate(
                                    input_variables=['capital'],
                                    template="tell me a famous place from {capital}"
)

famous_chain= LLMChain(llm=llm,prompt=famous_template,output_key="places")


In [ ]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("India")

### Sequential Chain

In [ ]:
capital_prompt = PromptTemplate(
    input_variables=['country'],
    template="tell me the capital of {country}"
    )
capital_chain= LLMChain(llm=llm,prompt=capital_prompt,output_key="capital")


In [ ]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains=[capital_chain,famous_chain],
input_variables = ['country'],
output_variables = ['capital','places'])
chain({"country":"India"})

## Chat Models using OpenAI

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage


In [ ]:
chatllm= ChatOpenAI(openai_api_key=openai_api_key,temperature=0.6,model="gpt-3.5-turbo")

In [ ]:
chatllm

In [ ]:
chatllm([
    SystemMessage(content="You are an intelligent AI assistant, you are working as a Insurance agent chatbot which helps user in insurance services"),
    HumanMessage(content="I want to make a payment for my car Insurance")
])

## Prompt Template + LLM + Output Parser

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [ ]:
class CommaSeparatedOutput(BaseOutputParser):
    def parse(Self,text:str):
        return text.strip().split(",")

In [ ]:
template = "You are helpful assistant.when the user given any input, you should answer it in according to the user's question "
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([("system",template),("human",human_template)])


In [ ]:
chain = chatprompt| chatllm | CommaSeparatedOutput()

In [ ]:
chain.invoke({"text":"Who is Highest scorer in Test Cricket?"})